In [1]:
import pandas as pd
import re

# TSV 파일 읽기
with open("review.sorted.uniq.preprocessing.tok.bpe.tsv", 'r') as f:
    lines = f.readlines()

# 데이터를 저장할 리스트 초기화
data = []
labels = []

# 각 줄을 처리
for line in lines:
    # 탭(\t)으로 구분
    split_line = line.strip().split('\t')
    
    # 첫 번째 열: 레이블 처리
    if split_line[0].lower() == 'positive':
        labels.append(1)
    elif split_line[0].lower() == 'negative':
        labels.append(0)
    else:
        raise ValueError(f"Unknown label: {split_line[0]}")
    data.append(split_line[1])

In [13]:
## 리뷰 벡터화
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer 객체 생성
count_vectorizer = CountVectorizer()

# sentences 데이터에 대한 피처 변환 수행
# sentences는 분석할 텍스트 데이터의 리스트

# neg는 앞에, pos는 뒤에 있기에 이를 합침
# 30만개는 너무 커서 한번에 안담긴다...
bag_size = 20000
data_mini = data[:bag_size]+data[-bag_size:]
labels_mini = labels[:bag_size] +labels[-bag_size:]
features = count_vectorizer.fit_transform(data_mini)
print(f"document 수: {features.shape[0]}")
print(f"단어수: {features.shape[1]}")

# features 객체를 NumPy 배열로 변환
vectorized_sentences = features.toarray()

document 수: 40000
단어수: 18942


In [14]:
feature_names = count_vectorizer.get_feature_names_out()
# 벡터화된 문장과 피처 이름을 이용해 DataFrame 생성
df = pd.DataFrame(vectorized_sentences, columns=feature_names)

# 데이터프레임의 인덱스 이름 지정
df.index.name = 'sentence'

## 원핫 인코딩

In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# 문장으로 부터 상위 100 개 단어로 vocabulary 작성
tokenizer = Tokenizer(num_words=15000, oov_token='<OOV>')
# sentences에 포함된 문장들을 기반으로 단어의 토큰화를
# 수행하며, 각 단어에 고유한 인덱스를 할당
tokenizer.fit_on_texts(data_mini)

# sentences 데이터를 시퀀스로 변환
sequences = tokenizer.texts_to_sequences(data_mini)

# 시퀀스에 패딩 적용 (문장의 뒤쪽을 패딩하고, 필요시 뒤쪽을 잘라냄)
padded = pad_sequences(sequences, padding='post', truncating='post')

In [ ]:
import reservoirpy as rpy
from reservoirpy.nodes import Reservoir
from reservoirpy.nodes import Reservoir, Ridge, Input
import numpy as np

ran = 10000
# 단순한 +이어붙이기는 오류가 발생한다, padded는 2차원이기 때문이다
X_train = np.concatenate([padded[:ran], padded[-ran:]], axis=0)
Y_train = labels_mini[:ran] + labels_mini[-ran:]
X_test = np.concatenate([padded[ran+1:ran+500], padded[-ran-500:-ran-1]], axis=0)
Y_test = labels_mini[ran+1:ran+500] + labels_mini[-ran-500:-ran-1]

source = Input()
reservoir = Reservoir(100, sr=0.9, lr=0.1)
readout = Ridge( ridge=1e-6)

model = source >> reservoir >> readout

In [6]:
from IPython.display import clear_output

states_train = []
for x in X_train:  # 진행바를 숨김
    states = reservoir.run(x, reset=True)
    states_train.append(states[-1, np.newaxis])
    clear_output(wait=True)  # 출력 초기화

Running Reservoir-0: 100%|██████████| 1/1 [00:00<00:00, 1003.66it/s]


In [7]:
readout.fit(states_train, Y_train)

'Ridge-0': Ridge(ridge=1e-06, input_bias=True, in=500, out=1)

In [12]:
Y_pred = []
for x in X_test:
    states = reservoir.run(x, reset=True)
    y = readout.run(states[-1, np.newaxis])
    Y_pred.append(y)
    clear_output(wait=True)  # 출력 초기화

Running Ridge-0: 100%|██████████| 1/1 [00:00<?, ?it/s]


In [9]:
from sklearn.metrics import accuracy_score

threshold = 0.5

Y_pred_class = [1 if y_p[0] >= threshold else 0 for y_p in Y_pred]
Y_test_class = [y_t for y_t in Y_test]

score = accuracy_score(Y_test_class, Y_pred_class)

print("Accuracy: ", f"{score * 100:.3f} %")
score

Accuracy:  65.130 %


0.6513026052104208

In [11]:
for y_p in Y_pred:
    print(y_p)
for y_t in Y_test:
    print(y_t)
print(Y_test[0])

[[0.59530781]]
[[0.61632278]]
[[0.40104283]]
[[0.63364749]]
[[0.50210501]]
[[0.39913172]]
[[0.72281248]]
[[0.33347153]]
[[0.69559755]]
[[0.64823247]]
[[0.62811227]]
[[0.28503261]]
[[0.67821399]]
[[0.8297839]]
[[0.75125237]]
[[0.67766868]]
[[0.86844968]]
[[0.65204883]]
[[0.59214104]]
[[0.27399902]]
[[0.73298617]]
[[0.45171247]]
[[0.62828979]]
[[0.85863131]]
[[0.68785991]]
[[0.92206118]]
[[0.45229366]]
[[0.73625152]]
[[0.74620122]]
[[0.84957638]]
[[0.60499119]]
[[0.78183005]]
[[0.49611497]]
[[0.50970471]]
[[0.59265716]]
[[0.79204102]]
[[0.82107997]]
[[0.63498877]]
[[0.59032164]]
[[0.83385174]]
[[-0.12829577]]
[[0.67050816]]
[[0.62161882]]
[[0.51875589]]
[[0.02858017]]
[[0.61725066]]
[[0.07112605]]
[[0.36343483]]
[[0.54879854]]
[[0.51164732]]
[[0.21725439]]
[[0.73464196]]
[[0.39517385]]
[[0.29821842]]
[[0.76325515]]
[[0.81821336]]
[[0.32625904]]
[[0.7495269]]
[[0.65994858]]
[[0.72804921]]
[[0.68122095]]
[[0.71104543]]
[[0.34915565]]
[[0.62649052]]
[[0.70819983]]
[[0.72640152]]
[[0.8272460